In [1]:
# !pip install langchain-chroma

## Import libraries

In [2]:
import os

os.environ["HF_HOME"] = "/space/hotel/phit/personal/experiments/weights"
os.environ["TORCH_HOME"] = "/space/hotel/phit/personal/experiments/weights"

from typing import List, Optional, Union

from langchain.callbacks import FileCallbackHandler
from langchain.retrievers import ContextualCompressionRetriever, ParentDocumentRetriever
from langchain.retrievers.document_compressors import EmbeddingsFilter
from langchain.storage import InMemoryStore
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import UnstructuredFileLoader, JSONLoader
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.vectorstores import FAISS , Chroma
# from langchain_chroma import Chroma
from langchain_core.documents import Document
from loguru import logger
from rich import print
from sentence_transformers import CrossEncoder

from unstructured.cleaners.core import clean_extra_whitespace, group_broken_paragraphs

logfile = "log/output.log"
logger.add(logfile, colorize=True, enqueue=True)
handler = FileCallbackHandler(logfile)


persist_directory = None

In [3]:
from langchain_community.chat_models import ChatOllama


## Utility functions

In [4]:
VIETNAMWORKS = ['_id', 'url', 'job_name', 'company_name', 'salary', 'end_date',
       'address', 'posted_date', 'job_function', 'job_industry', 'job_level',
       'skill', 'preferred_language', 'job_description'] # job_requirements

TOPCV = ['_id', 'urls', 'job_name', 'company_name', 'address', 'salary',
       'remaining', 'job_description', 'benefits',
       'application_method', 'level', 'experience', 'number_of_recruitment',
       'work_form', 'gender', 'working_time'] # job_requirements

class RAGException(Exception):
    """
    Custom exception class for RAG (Retriever, Answerer, Generator) module.
    """

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)


def load_pdf(
    files: Union[str, List[str]] = "../data/cv/Bui Tien Phat resume (1).pdf"
) -> List[Document]:
    """
    Load PDF files and return a list of Document objects.

    Args:
        files (Union[str, List[str]], optional): Path(s) to the PDF file(s) to be loaded. Defaults to "../data/cv/Bui Tien Phat resume (1).pdf".

    Returns:
        List[Document]: A list of Document objects representing the loaded PDF files.
    """
    if isinstance(files, str):
        loader = UnstructuredFileLoader(
            files,
            post_processors=[clean_extra_whitespace, group_broken_paragraphs],
        )
        return loader.load()

    loaders = [
        UnstructuredFileLoader(
            file,
            post_processors=[clean_extra_whitespace, group_broken_paragraphs],
        )
        for file in files
    ]
    docs = []
    for loader in loaders:
        docs.extend(
            loader.load(),
        )
    return docs


# Define the metadata extraction function.
def metadata_func(record: dict, metadata: dict) -> dict:
    """
    Process the record dictionary and update the metadata dictionary with the relevant values.

    Args:
        record (dict): The record dictionary containing the data to be processed.
        metadata (dict): The metadata dictionary to be updated.

    Returns:
        dict: The updated metadata dictionary.
    """
    
    # metadata["urls"] = record.get("urls")
    # metadata["job_name"] = record.get("job_name")
    # metadata["company_name"] = record.get("company_name")
    # metadata["address"] = record.get("address")
    # metadata["salary"] = record.get("salary")
    # metadata["remaining"] = record.get("remaining")
    # # metadata[""] = record.get("Mô tả công việc")
    # metadata["Yêu cầu ứng viên"] = record.get("Yêu cầu ứng viên")
    # metadata["Quyền lợi"] = record.get("Quyền lợi")
    # metadata["Địa điểm làm việc"] = record.get("Địa điểm làm việc")
    # metadata["Cách thức ứng tuyển"] = record.get("Cách thức ứng tuyển")
    # metadata["Cấp bậc"] = record.get("Cấp bậc")
    # metadata["Kinh nghiệm"] = record.get("Kinh nghiệm")
    # metadata["Số lượng tuyển"] = record.get("Số lượng tuyển")
    # metadata["Hình thức làm việc"] = record.get("Hình thức làm việc")
    # metadata["Giới tính"] = record.get("Giới tính")
    
    for key in VIETNAMWORKS:
        metadata[key] = record.get(key)
    
    metadata = {key: f'{" ".join(val[0]) if isinstance(val, list) else val}' for key, val in metadata.items()}

    

    return metadata


def load_jsonl(
    files: Union[str, List[str]] = "../data/crawl/train_test.jsonl"
) -> List[Document]:
    """
    Load JSONL files and return a list of documents.

    Args:
        files (Union[str, List[str]], optional): Path or list of paths to the JSONL files. 
            Defaults to "../data/crawl/train_test.jsonl".

    Returns:
        List[Document]: A list of Document objects.

    """
    if isinstance(files, str):
        loader = JSONLoader(
            files,
            json_lines=True,
            jq_schema='.',
            content_key="job_requirements", 
            text_content=False,
            metadata_func=metadata_func
        )
        return loader.load()

    loaders = [
        JSONLoader(
            file,
            json_lines=True,
            jq_schema='.messages[]',
            content_key="content",
            metadata_func=metadata_func
        )
        for file in files
    ]
    docs = []
    for loader in loaders:
        docs.extend(
            loader.load(),
        )
    return docs

## RAG Components

In [5]:
def rerank_docs(reranker_model, query, retrieved_docs):
    """
    Reranks the retrieved documents based on a given reranker model and query.

    Args:
        reranker_model: The reranker model used for scoring the documents.
        query: The query used for reranking the documents.
        retrieved_docs: A list of retrieved documents.

    Returns:
        A list of tuples containing the retrieved documents and their corresponding scores,
        sorted in descending order of scores.
    """
    query_and_docs = [(query, r.page_content) for r in retrieved_docs]
    scores = reranker_model.predict(query_and_docs)
    return sorted(list(zip(retrieved_docs, scores)), key=lambda x: x[1], reverse=True)

def create_parent_retriever(
    docs: List[Document], embeddings_model: HuggingFaceBgeEmbeddings()
):
    """
    Creates a parent document retriever using the given list of documents and embeddings model.

    Args:
        docs (List[Document]): A list of documents to be added to the retriever.
        embeddings_model (HuggingFaceBgeEmbeddings): The embeddings model to be used for indexing.

    Returns:
        ParentDocumentRetriever: The created parent document retriever.

    """
    parent_splitter = RecursiveCharacterTextSplitter(
        separators=["\n\n\n", "\n\n"],
        chunk_size=2000,
        length_function=len,
        is_separator_regex=False,
    )

    # This text splitter is used to create the child documents
    child_splitter = RecursiveCharacterTextSplitter(
        separators=["\n\n\n", "\n\n"],
        chunk_size=500,#1000,
        chunk_overlap=150,#300,
        length_function=len,
        is_separator_regex=False,
    )
    # The vectorstore to use to index the child chunks
    vectorstore = Chroma(
        collection_name="split_documents",
        embedding_function=embeddings_model,
        persist_directory=persist_directory,
    )#.as_retriever()
    
    print("vectorstore: ", vectorstore)
    # The storage layer for the parent documents
    store = InMemoryStore()
    retriever = ParentDocumentRetriever(
        vectorstore=vectorstore,
        docstore=store,
        child_splitter=child_splitter,
        parent_splitter=parent_splitter,
        k=10,
    )
    retriever.add_documents(docs)
    return retriever


def retrieve_context(query, retriever, reranker_model):
    """
    Retrieves relevant documents based on the given query using a retriever and reranks them using a reranker model.

    Args:
        query (str): The query to retrieve relevant documents for.
        retriever: The retriever object used to retrieve relevant documents.
        reranker_model: The reranker model used to rerank the retrieved documents.

    Returns:
        list: A list of reranked documents.

    Raises:
        RAGException: If no relevant documents are retrieved with the given query.
    """
    retrieved_docs = retriever.get_relevant_documents(query)

    if len(retrieved_docs) == 0:
        raise RAGException(
            f"Couldn't retrieve any relevant document with the query `{query}`. Try modifying your question!"
        )
    reranked_docs = rerank_docs(
        query=query, retrieved_docs=retrieved_docs, reranker_model=reranker_model
    )
    return reranked_docs

# https://github.com/FlagOpen/FlagEmbedding/tree/master/FlagEmbedding/BGE_M3
def load_embedding_model(
    model_name: str = "BAAI/bge-large-en-v1.5", device: str = "cuda"
) -> HuggingFaceBgeEmbeddings:
    """
    Load the embedding model for text encoding.

    Args:
        model_name (str): The name of the pre-trained model to load. Defaults to "BAAI/bge-large-en-v1.5".
        device (str): The device to use for model inference. Defaults to "cuda".

    Returns:
        HuggingFaceBgeEmbeddings: The loaded embedding model.

    """
    model_kwargs = {"device": device}
    encode_kwargs = {
        "normalize_embeddings": True
    }  # set True to compute cosine similarity
    embedding_model = HuggingFaceBgeEmbeddings(
        model_name=model_name,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs,
    )
    return embedding_model

# https://github.com/FlagOpen/FlagEmbedding/tree/master/FlagEmbedding/llm_reranker
def load_reranker_model(
    reranker_model_name: str = "BAAI/bge-reranker-large", device: str = "cuda"
) -> CrossEncoder:
    """
    Loads a reranker model for ranking search results.

    Args:
        reranker_model_name (str): The name or path of the reranker model to be loaded.
        device (str): The device to be used for running the model (default: "cuda").

    Returns:
        CrossEncoder: The loaded reranker model.
    """
    reranker_model = CrossEncoder(
        model_name=reranker_model_name, max_length=512, device=device, 
    )
    return reranker_model


# def main(
#     file: str = "../data/crawl/train_test.jsonl",
#     query: Optional[str] = None,
#     llm_name="mistral",
# ):
#     # docs = load_pdf(files=file)
#     docs = load_jsonl("../data/crawl/vnw.jsonl")
#     # print(docs)

#     embedding_model = load_embedding_model()
#     retriever = create_parent_retriever(docs, embedding_model)
#     reranker_model = load_reranker_model()

#     context = retrieve_context(
#         query, retriever=retriever, reranker_model=reranker_model
#     )[0]
#     print("context:\n", context, "\n", "=" * 50, "\n")


# if __name__ == "__main__":
#     # from jsonargparse import CLI

#     # CLI(main)
#     # main(query="What is the job description for Network Engineer?")
#     main(query="What is job description in Da Nang ?")
    

In [6]:
import os
files = ["../data/cv/" + file for file in os.listdir("../data/cv")]
print(files)

cv = load_pdf(files)

[
    '../data/cv/CV Cap Tan Dat - Intern Developer.pdf',
    '../data/cv/CV_SonBao_DS.pdf',
    '../data/cv/Bui Tien Phat resume (1).pdf',
    '../data/cv/CV Nguyễn việt hoàng - CV lập trình-TopCV.vn.pdf',
    '../data/cv/CV _ Đinh Tuấn Nam_BE Developer(.NET)_HCM.pdf',
    '../data/cv/CV_MaiHuyKhang_DA.pdf',
    '../data/cv/CV_NGUYEN_MINH_SON.pdf'
]

In [10]:
print(cv[2])

Document(
    page_content='Data Science/AI Engineer\n\nTIEN PHAT BUI\n\n0962980173 | buitienphat2462002@gmail.com 
https://www.linkedin.com/in/tien-phat-bui-ba3421222/ https://github.com/ShynBui/ Go Vap district, Ho Chi Minh 
city\n\nEducation\n\nB ACH EL OR OF IN FOR M ATION TECH N OL O G Y(Expected 05/2024) – Ho Chi Minh City Open 
University – 97 Vo Van Tan, Ward 6, District 3, Ho Chi Minh City GPA: 3.76/4.00 (Highest in Major) Majors: 
Information Technology Relevant Coursework: Machine Learning, Data Visualization and Communication, Database 
Systems and SQL, Natural Language Processing, Computer Vision, Deep Learning and Neural 
Networks\n\nSkills\n\nPython Libraries and frameworks: pandas, matplotlib, numpy, BeutifulSoup4, re, os, 
scikit-learn, Hugging Face, transformers, PyTorch, Python-Flask, LangChain, NLTK\n\nBig Data Apache Hadoop, 
PySpark\n\nSQL Software: SQLite, MySQL, SQL, PostgreSQL\n\nPower Bi Tools: Power Query, Visualize\n\nC++ Libraries:
STL, Boot C++, Armadillo\n\nProjects\n\nLEGAL SEAR CH AN D R ES PON SE PLATFOR M – Group competition 
project\n\nNovember 2023 \n\n-\n\n now\n\nProvides users with quick access to legal information, delivers reliable 
answers to legal queries, and enables efficient navigation\n\nthrough legal codes and documents\n\nUtilizes ReactJS
for front-end development, employs Python Flask for back-end functionality, and enhances its capabilities 
with\n\ncomplementary integration tools such as CI/CD (Continuous Integration/Continuous Deployment) and Docker for
streamlined deployment processes, ensuring efficient and reliable operation\n\nProject management, ensuring a 
waterfall system development process, data search and preprocessing, training deep learning\n\nmodels based on 
PhoBERT and LangChain for question-answering tasks for answering legal and training another model to classify 
"legal definition" sentences in legal documents, taking on the role of data designer using MySQL, and visualize the
model training process for reporting using matplotlib\n\nWinning third prize in the Vietnam Student Informatics 
Olympiad - Open source competition, the project attracted the jury for its creativity and was considered to have 
the most potential. The product is open source code that is being invested in, developed and raised capital The 
article introduces the project: 
https://vfossa.vn/tin-tuc/code-heroes-nen-tang-tra-cuu-va-giai-dap-phap-luat-ung-dung-sang- 
tao-cac-mo-hinh-ngon-ngu-lon-698.html\n\n\n\nAN AUTOM ATICALLY AN S WER IN G EN GLISH AN D VIETN AM ESE R EA D IN G
COM PR EH EN SION S YSTEM – Personal June 2021 - January 2022 Project\n\nAssisting users in automatically answering
Vietnamese or English reading comprehension questions by providing the system with\n\na passage of information 
(provided by the user), from which the system extracts information to answer the user\'s queries ● Using Hugging 
Face as a model repository, we employ a BERT-based model for answering English reading 
comprehension\n\n\n\n\n\nquestions and a PhoBERT-based model for answering Vietnamese reading comprehension 
questions. Gradio is utilized to design an interactive interface for user interaction Taking on the role of data 
preparation, I utilize available datasets and scrape data from the web using BeautifulSoup4. Data processing is 
performed using Python, while model fine-tuning is carried out through the use of Transformers and PyTorch. Model 
evaluation and visualization of evaluation results are conducted using matplotlib The English question-answering 
model achieved a significant 78% accuracy when applied to multiple-choice questions. The Vietnamese 
question-answering model also achieved an exact match accuracy of 72%\n\nOTH ER PR OJECT\n\nOnline library ● ● 
Website for book sales management ● Vietnamese toxic comment classification Achievements and Awards\n\nTime 
management website\n\nTH E 2022 ICPC ASI A H O CH I M INH CITY – HCMUTE – Ho Chi Minh City\n\nDecember 
2022\n\nRanked 

In [15]:
import spacy

def load_model(model_path):
    nlp = spacy.load(model_path)
    return nlp

def predict_entities(text, nlp):
    doc = nlp(text)
    entities = []
    for ent in doc.ents:
        entities.append((ent.text, ent.label_))
    return entities

# Load the trained model
model_path = "/space/hotel/phit/personal/applied-data-science/src/applied_data_science/nlp_ner_model"
nlp = load_model(model_path)

# Example usage
# text = "I work at GitHub and live in San Francisco."

text = cv[0].page_content
entities = predict_entities(text, nlp)
for entity, label in entities:
    print("Entity:", entity)
    print("Label:", label)
    print()

Entity: Cap Tan Dat

Label: Name

Entity: Web developer intern

Label: Designation

Entity: Ho Chi Minh City Open University

Label: Degree

Entity: 2020

Label: Graduation Year

Entity: Programming languages

Other

Communication

Label: Skills

In [ ]:


docs = load_jsonl("../data/crawl/vnw.jsonl")

embedding_model = load_embedding_model()
retriever = create_parent_retriever(docs, embedding_model)
reranker_model = load_reranker_model()

context = retrieve_context(
    cv, retriever=retriever, reranker_model=reranker_model
)[0]
print("context:\n", context, "\n", "=" * 50, "\n")

In [17]:
import pandas as pd
df = pd.read_json("../data/crawl/vnw.jsonl", lines=True)
df

,_id,url,job_name,company_name,salary,end_date,address,posted_date,job_function,job_industry,job_level,skill,preferred_language,job_description,job_requirements
0,661621edbb86ac211a66e91a,https://www.vietnamworks.com/giang-vien-cong-n...,Giảng Viên Công Nghệ Thông Tin (Swinburne),Đại Học FPT TP.HCM,Thương lượng,2024-04-16,"Lô E2a-7, Đường D1, Khu Công Nghệ Cao, Long Th...",04/03/2024,"Công Nghệ Thông Tin/Viễn Thông, > , System/Cl...",Giáo dục/Đào Tạo,Nhân viên,"Giáo Dục, Công Nghệ Thông Tin, Khoa Học Máy Tí...",Bất kỳ,[[- Giảng dạy/nghiên cứu và các hoạt động chuy...,[[- Tối thiểu tốt nghiệp cao học lãnh vực giản...
1,661621edbb86ac211a66e91c,https://www.vietnamworks.com/it-infrastructure...,IT Infrastructure Operations Expert,Navigos Search,$2000 - $3000,2024-04-19,Hanoi / HCMC,20/03/2024,"Công Nghệ Thông Tin/Viễn Thông, > , Phần Cứng...",Tài Chính,Nhân viên,AWS Docker,Bất kỳ,"[[- To prevent the incidents & problems, - Man...",[[- A bachelor’s degree in computer science or...
2,661621edbb86ac211a66e927,https://www.vietnamworks.com/junior-backend-de...,Junior Backend Developer,Codeforce VINA,Thương lượng,2024-04-11,"7th floor Saigonbus building, 39 Hai Thuong La...",12/03/2024,"Công Nghệ Thông Tin/Viễn Thông, > , Phần Mềm ...",Phần Mềm CNTT/Dịch vụ Phần mềm,Nhân viên,"MySQL, Typescript, RESTful API Design, Lập Trì...",Bất kỳ,[[Job Description: We are seeking a motivated ...,"[[***Job Requirements:, • Proficiency in backe..."
3,661621edbb86ac211a66e928,https://www.vietnamworks.com/senior-qc-enginee...,Senior QC Engineer,Globee Software & E-Commerce,Tới $1000,2024-04-17,"No. 3 Ba Thang Hai, Ward 11, District 10",18/03/2024,"Công Nghệ Thông Tin/Viễn Thông, > , QA/QC/Sof...",Phần Mềm CNTT/Dịch vụ Phần mềm,Nhân viên,"Ecommerce Platforms, Salesforce, Ecommerce Tes...",Tiếng Anh,"[[- Understand the system, analyze the documen...",[[- At least 03 YOE working as QC (auto/manual...
4,661621edbb86ac211a66e92a,https://www.vietnamworks.com/qm-staff-japanese...,"QM Staff (Japanese N2+, Open for IT Comtor)",NEC Vietnam,Thương lượng,2024-04-17,"12th Floor, Gelex Tower, 52 Le Dai Hanh, Hai B...",18/03/2024,"Công Nghệ Thông Tin/Viễn Thông, > , QA/QC/Sof...",Phần Mềm CNTT/Dịch vụ Phần mềm,Nhân viên,"QMS, QMS Management, IT Auditing, IT Risk Mana...",Bất kỳ,"[[Your role & responsibilities:, NECVN perform...","[[Your skills & qualifications:, - Bachelor's ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1349,6628aee1ebef1caba803300a,https://www.vietnamworks.com/business-analyst-...,Business Analyst,ELCA Information Technology (Vietnam) Limited,Thương lượng,2024-05-23,"15F-19F CII Tower, 152 Dien Bien Phu, Ward 25,...",23/04/2024,"Công Nghệ Thông Tin/Viễn Thông, > , Phân Tích...",Phần Mềm CNTT/Dịch vụ Phần mềm,Nhân viên,"SQL, Business Analysis, UI/UX, UI Mockup, Func...",Tiếng Anh,"[[MISSION], [You will work alongside developer...","[[QUALIFICATIONS], [Required], [- Minimum of 3..."
1350,6628aee1ebef1caba8033010,https://www.vietnamworks.com/network-engineer-...,Network Engineer,"K2 SOLUTIONS CO., LTD.",Thương lượng,2024-05-23,,23/04/2024,"Công Nghệ Thông Tin/Viễn Thông, > , System/Cl...",Hệ thống CNTT & Thiết bị,Nhân viên,"Network, CCNP, CCNA, Data Center Management, N...",Tiếng Anh,"[[ABOUT THE POSITION:], [You will be part of K...","[[POSTION REQUIREMENT], [1.\tNetwork Design an..."
1351,6628aee1ebef1caba8033011,https://www.vietnamworks.com/embedded-software...,"Embedded Software Engineer (Autosar, CAN Tools...",Ban Vien Corporation,Thương lượng,2024-05-23,"Ban Vien Tower, 54-56-58 Street 2, Van Phuc Re...",23/04/2024,"Công Nghệ Thông Tin/Viễn Thông, > , Phần Mềm ...",Phần Mềm CNTT/Dịch vụ Phần mềm,Nhân viên,"C/C++, Embedded Software, AUTOSAR, Software En...",Tiếng Anh,[[- Software development for Automotive Domain...,"[[* MUST HAVE], [- Strong in C/C++ programming..."
1352,6628aee1ebef1caba803301d,https://www.vietnamworks.com/it-project-coordi...,IT Project Coordinator,RMIT University Vietnam,$1200 - $1400,2024-05-23,"702 Ngu

### Query schema

In [ ]:
# from typing import Optional

# from langchain_core.pydantic_v1 import BaseModel, Field


# class Search(BaseModel):
#     """Search over a database of tutorial videos about a software library."""

#     query: str = Field(
#         ...,
#         description="Similarity search query applied to video transcripts.",
#     )
#     publish_year: Optional[int] = Field(None, description="Year video was published")

### Query generation

In [ ]:
# from langchain_core.prompts import ChatPromptTemplate
# from langchain_core.runnables import RunnablePassthrough
# from langchain_openai import ChatOpenAI

# system = """You are an expert at converting user questions into database queries. \
# You have access to a database of tutorial videos about a software library for building LLM-powered applications. \
# Given a question, return a list of database queries optimized to retrieve the most relevant results.

# If there are acronyms or words you are not familiar with, do not try to rephrase them."""
# prompt = ChatPromptTemplate.from_messages(
#     [
#         ("system", system),
#         ("human", "{question}"),
#     ]
# )
# llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
# structured_llm = llm.with_structured_output(Search)
# query_analyzer = {"question": RunnablePassthrough()} | prompt | structured_llm

In [22]:
from langchain_core.output_parsers import StrOutputParser
import time

In [4]:
import os
from langchain_community.chat_models import ChatOllama

llm = ChatOllama(model="mistral", base_url="http://172.16.87.75:11434")


llm.invoke("What is the job description for Network Engineer?")

AIMessage(content=" A Network Engineer is responsible for designing, building, testing, and maintaining computer networks, such as Local Area Networks (LAN), Wide Area Networks (WAN) and Intranets. Here are some key responsibilities and tasks involved:\n\n1. Designing and implementing network architecture: This involves planning the layout of a company's network, considering factors like bandwidth needs, security requirements, and scalability.\n\n2. Network configuration and management: Configuring network devices like routers, switches, firewalls, and servers to ensure they operate efficiently and securely.\n\n3. Troubleshooting network issues: Identifying and resolving problems that may affect the network's performance or security, such as connectivity issues, network congestion, or unauthorized access attempts.\n\n4. Network monitoring and maintenance: Monitoring network performance to ensure optimal operation, performing regular software updates and patches, and implementing preven

In [1]:
from ollama import Client 
import os

os.environ["NO_PROXY"] = "172.16.87.75"

ollama_client = Client(host="http://172.16.87.75:11434")
reponse = ollama_client.chat(model="mistral", messages=[
    {"role": "system", "content": "You are an expert at converting user questions into database queries. You have access to a database of tutorial videos about a software library for building LLM-powered applications. Given a question, return a list of database queries optimized to retrieve the most relevant results. If there are acronyms or words you are not familiar with, do not try to rephrase them."},
    {"role": "user", "content": "What is the job description for Network Engineer?"}
], options={"structured_output": {"type": "Search"}})

In [2]:
reponse

{'model': 'mistral',
 'created_at': '2024-06-10T05:35:49.755469409Z',
 'message': {'role': 'assistant',
  'content': ' To retrieve the job descriptions associated with the role "Network Engineer" from the tutorial videos database, here are some possible SQL queries:\n\n1. If there is a column named `role` in the table that stores video metadata:\n\n```sql\nSELECT description\nFROM tutorial_videos\nWHERE role = \'Network Engineer\';\n```\n\n2. If there isn\'t a specific column for the job description, but instead, the descriptions are included in the `content` or `transcript` columns, you can use full-text search features if available:\n\n```sql\n-- MySQL Full-Text Search\nSELECT id, content\nFROM tutorial_videos\nWHERE MATCH (content, transcript) AGAINST (\'Network Engineer\' IN BOOLEAN MODE);\n\n-- PostgreSQL using ts_query\nSELECT id, content, transcript\nFROM tutorial_videos\nWHERE to_tsvector(\'english\', content || \' \' || transcript) @@ to_tsquery(\'Network Engineer\');\n```'},


In [3]:
import os
files = ["../data/cv/" + file for file in os.listdir("../data/cv")]
print(files)

cv = load_pdf(files)

['../data/cv/CV Cap Tan Dat - Intern Developer.pdf',
 '../data/cv/CV_SonBao_DS.pdf',
 '../data/cv/Bui Tien Phat resume (1).pdf',
 '../data/cv/CV Nguyễn việt hoàng - CV lập trình-TopCV.vn.pdf',
 '../data/cv/CV _ Đinh Tuấn Nam_BE Developer(.NET)_HCM.pdf',
 '../data/cv/CV_MaiHuyKhang_DA.pdf',
 '../data/cv/CV_NGUYEN_MINH_SON.pdf']

In [9]:
print(cv[3].page_content)

HỒ SƠ CÁ NHÂN

 19/01/2003

 0387 36 1513

✉ hoangpham19112002@gmail.com

 https://github.com/hoangphampvh/

csharp5.git

 Nguyễn Cơ Thạch/Mỹ Đình 2/ Hà Nội

CAREER GOALS

Các mục tiêu ngắn hạn:

Củng cố, bổ sung những kiến thức hiện

có để hoàn thiện và nâng cao giá trị cá

nhân.

Đồng thời, trau dồi các kỹ năng và kiến

thức .NET hiện tại, làm việc trong vai

trò lập trình viên .NET.

Mục tiêu dài hạn:

Trở thành Nhà phát triển cấp cao có

chuyên môn vững vàng trong vòng 3-4

năm kinh nghiệm chuyên môn.

SKILLS

Kỹ năng cứng Đọc và hiểu tài liệu tiếng Anh GitHub, ASP .NET Core, .NET framework, SOLID, design pattern, Redis, 
Bootstrap và MudBlazor Sử dụng khung Bootstrap và MudBlazor để thiết kế giao diện người dùng. HTML, CSS và jQuery 
(KENDO) cho Front-end Kiến thức cơ bản về kiểm thử ứng dụng.

Kỹ năng mềm Làm việc hiệu quả trong các dự án nhóm với kỹ năng hợp tác. Duy trì sự tập trung và kiên trì trong công
việc dưới áp lực và thời hạn

NGUYỄN VIỆT HOÀNG FREHER .NET PROGRAMMER

HỌC VẤN

FPT Polytechnic College

NFORMATION TECHNOLOGY 10/2021 

-

 01/2024

Hiện tại tôi đang học học kỳ thứ 7 với tư cách là sinh viên. Tôi đã đạt được danh hiệu Sinh viên Xuất sắc trong học
kỳ Mùa thu năm 2021 và giành được ba danh

hiệu Sinh viên Xuất sắc cho các học kỳ Mùa xuân 2022, Mùa hè 2022 và Mùa thu

2022.

KINH NGHIỆM LÀM VIỆC

FPT Polytechnic College

.NET PROGRAMMER 10/2021 

-

 01/2024

Thực hiện các dự án liên quan đến .NET.

Cong ty xxx(3 tháng)

THỰC HIỆN CÁC DỰ ÁN LIÊN QUAN ĐẾN .NET. 24/5 - 24/8

Làm quen và tham gia mô hình phát triển hệ thống phần mềm sau 3 tháng thực tập.

FPT Project Workshop (2 tháng)

DEVELOPER 01/04/2023 

-

 20/6/2023

Mô tả dự án: Phục vụ nhu cầu tổ chức các cuộc thi đánh máy cho học sinh, sinh viên Công nghệ được sử dụng:

Sử dụng EF Core để truy cập dữ liệu. Sử dụng LinQ để truy vấn.

Xây dựng giao diện người dùng bằng Bootstrap và Blazor.

Mã nguồn được quản lý bằng GitHub. Đóng góp chính cho dự án:

Xây dựng biểu đồ kết quả thống kê cho các cuộc thi.

Đã triển khai ủy quyền người dùng và Jwt.

My Project

-

Mô tả dự án: Xây dựng website thương mại điện tử.

Công nghệ được sử dụng:

chặt chẽ. Đảm bảo công việc được hoàn thành đúng thời gian và chất lượng cao. Quản lý thời gian hiệu quả để hoàn 
thành nhiệm vụ được giao và đạt được mục tiêu. Chủ động trong công việc và học hỏi từ người khác. Chấp nhận những 
lời chỉ trích và phản hồi để cải thiện hiệu suất công việc.

Đã sử dụng ASP.NET Core, Blazor, Bootstrap, Git và máy chủ Redis.

Chức Năng: Xác thực và ủy quyền người dùng (Danh tính).

Chức năng nhập và xuất Excel.

Đăng nhập bằng JWT và OAuth2.

Xử lý thanh toán bằng chứng từ. Thông báo email tự động.

Mua và bán sản phẩm.

Tích hợp SignalR cho các tính năng thời gian thực.

Tích hợp thanh toán VnPay, MoMo, ZaloPay

Kiến thức đang và đã học



Docker, RabbitMQ

© topcv.vn

In [3]:
original_list = [(1, 2), (3, 2)]
a, b = zip(*original_list)

print(list(a))  # Output: [1, 3]
print(list(b))  # Output: [2, 2]


[1, 3]
[2, 2]


In [11]:
# query = "What are some job descriptions for Data?"
query = cv[2].page_content
context = retrieve_context(
    query, retriever=retriever, reranker_model=reranker_model
)[:]
print("context:\n", context, "\n", "=" * 50, "\n")

context:

[
    (
        Document(
            page_content='"- Bachelors degree in Computer Science, Electrical Engineering, or Artificial 
Intelligence.", - At least 4+ years of experience in AI or relevant positions., - Have 2+ year of team leader 
experience or relevant positions., - Knowledge of the following areas (but not limited to): Recommendation system, 
- Computer Vision, NLP (information extraction), Large Language Models, OCR pipeline., - Knowledge of AI 
frameworks: Pytorch, Tensorflow, Triton, TF serving., - Experience in writing API to wrap AI models., - Ability to 
plan & organize AI services, especially in emergency situations., - Excellent problem-solving and analytical 
skills., - Be proactive, enthusiastic, hard-working., - Be integrity, careful, accurate., *** What we offer, - 
Working time from Monday to Friday (09:00 am - 18:00 pm)., - Good chance to study and develop career path stably 
with internal promotion and rotation., - 100% statutory insurance contribution as per the labor law., - Bao Viet 
Healthcare insurance., - 100% gross salary in probation., - 12 Annual Leave (AL) per year + extra 1 AL per extra 
working year., - Annual bonus: depending on business & individual performance., - Annual salary review., - ESOP 
allows you to financially share in our future success., - Other rewards: 5-star, employee of the month (EOTM), 
employee of the year (EOTY), loyalty employee (3-year, 5-year, 8-year, etc.), "- Internal Activities: General 
Meeting, Company Trip, International Womens Day,", "Vietnamese Womens Day Mid-Autumn Festival, Family Day, 
Christmas, Year-end Party, etc."',
            metadata={
                'source': 
'/mnt/net/i2x256-ai03/hotel/phit/personal/applied-data-science/data/preprocessed/preprocessed_vnw.jsonl',
                'seq_num': '555',
                '_id': '661621f5bb86ac211a66ee2e',
                'url': 
'https://www.vietnamworks.com/ai-manager-ocr-nlp-1759990-jv?source=searchResults&searchType=2&placement=1760090&sor
tBy=latest',
                'job_name': 'AI Manager (Ocr/nlp)',
                'company_name': 'Got It',
                'salary': '$3000 - $4000',
                'end_date': '2024-04-28',
                'address': '9-11 Nguyễn Văn Thủ, P. Đa Kao, Quận 1, TP. Hồ Chí Minh',
                'posted_date': '29/03/2024',
                'job_function': 'Công Nghệ Thông Tin/Viễn Thông,  > , Phần Mềm Máy Tính',
                'job_industry': 'Phần Mềm CNTT/Dịch vụ Phần mềm',
                'job_level': 'Trưởng phòng',
                'skill': 'TensorFlow, Computer Science, Application Development, Product Development, Pipeline 
Development',
                'preferred_language': 'Bất kỳ',
                'job_description': 'What you need to do:, - Lead and deliver AI solutions for Dayone, including 
(but not limited to): maximizing Got It voucher’s revenue (e.g. intelligent voucher redemption recommendation 
system), online shopping recommendation systems, AI-assisted productivity boosting applications, and OCR system for
invoices and business documents., - Work with large and real world data from our products (Got It, Buzz It, Scan 
It, and internal tools) to generate insightful information, assist business decisions, and maximize business value 
of our products., - Develop tools and frameworks to help training, deploying, and monitoring AI models more generic
and effective., - Organize AI team to optimize resources yet deliver high quality services., - Collaborate with 
other teams to assure the service operations meet expectations from internal and external users., - Perform 
additional duties as required by Line Manager.'
            }
        ),
        0.7402199
    ),
    (
        Document(
            page_content='1. REQUIREMENT, ● Educational: Graduated from University/College or higher, ● Work 
Experience: Around 5+ years of technical experience, ● Specialty/Qualification: Engineer/Bachelor or higher, ● 
Location: Viet Nam, ● Skills:, - Se

---

In [23]:
cv[0].page_content

"Cap Tan Dat\n\nWeb developer intern\n\n\uf095 0961414817 ✉ tandatkt002@gmail.com \uf0c1 https://github.com/Penz7\n\n\uf3c5 Ho Chi Minh City,Ward 7,Binh Thanh District\n\nOBJECTIVE\n\nObtain more knowledge about web programming by gaining hands-on experience in a formal working setting. For a better understanding of how to work in a professional setting and further your career, learn from your forebears, particularly from your\n\nelders at the organization. To ensure independence and top performance in your everyday work, learn time management skills, how to work efficiently, and how to\n\ndevelop an organized work schedule. Finally, I wish to have the chance to foster development and contribute to the company's future success and growth in addition to being\n\nhired on as a formal employee.\n\nEDUCATION\n\nHo Chi Minh City Open University\n\n2020 \n\n-\n\n 2024\n\nMajor: Information Technology GPA: 2.68\n\nExtracurricular Involvement: \n\n•\n\n Mobile device programing club\n\nEXPERIE

In [13]:
text_items = "Cap Tan Dat\n\nWeb developer intern\n\n\uf095 0961414817 ✉ tandatkt002@gmail.com \uf0c1 https://github.com/Penz7\n\n\uf3c5 Ho Chi Minh City,Ward 7,Binh Thanh District\n\nOBJECTIVE\n\nObtain more knowledge about web programming by gaining hands-on experience in a formal working setting. For a better understanding of how to work in a professional setting and further your career, learn from your forebears, particularly from your\n\nelders at the organization. To ensure independence and top performance in your everyday work, learn time management skills, how to work efficiently, and how to\n\ndevelop an organized work schedule. Finally, I wish to have the chance to foster development and contribute to the company's future success and growth in addition to being\n\nhired on as a formal employee.\n\nEDUCATION\n\nHo Chi Minh City Open University\n\n2020 \n\n-\n\n 2024\n\nMajor: Information Technology GPA: 2.68\n\nExtracurricular Involvement: \n\n•\n\n Mobile device programing club\n\nEXPERIENCE\n\nProgramming and Teamwork Skills\n\n\n\nDuring my time studying and cultivating in the university environment, I have formed the thinking and working style of a\n\nprogrammer, with work that requires perseverance, concentration and the ability to work with others partner. Along with that is the ability to exchange and speak in front of crowds with your own products, without being afraid of difficulties\n\nand challenges. Successfully complete industry foundation subjects such as: Data Structures & Algorithms, Object-Oriented Programming, Mobile\n\nProgramming, Software Testing,...\n\nPROJECTS\n\nLearning English App\n\n( Aug 20, 2023 \n\n-\n\n Sep 21, 2023 )\n\nLanguage\n\nKotlin\n\nDescription project\n\nBuilding an application that allows learners to use their registered accounts to study courses by topic.\n\nYour team size\n\n2\n\nYour responsibility in project\n\nI am in charge of developing the area for personal information, managing login registration for the system, putting the online chat system into place using GetStream.io, and integrating\n\nChatGPT's API to enable communication with AI bots.\n\nProject structure\n\n• Database: Firestore, Firebase storage, Firebase authentication \n\n• Architecture: MVVM (Built based on Google App standard architecture)\n\nGithub\n\nFindHostel\n\n( Jun 25, 2023 \n\n-\n\n Sep 21, 2023 )\n\nLanguage\n\nDescription project\n\nYour team size\n\nYour responsibility in project\n\nDatabase\n\nGithub\n\nWebsite Design Projects\n\n( Jan 16, 2022 \n\n-\n\n Sep 21, 2023 )\n\nProject\n\nEcommerce\n\n-\n\nDesign\n\n-\n\nWebsite\n\nDescription project (Ecommerce\n\n-\n\n Design\n\n-\n\nWebsite)\n\nClinic\n\n-\n\nSystem\n\nDescription project (Clinic\n\n-\n\nSystem)\n\nSKILLS\n\nProgramming languages\n\nOther\n\nCommunication\n\n• UI framework: Jetpack Compose\n\n• Support: GetStream.io (Chat API), Sapling AI (Grammar check), chatGPT\n\nhttps://github.com/TranDatk/LearningEngishApplication\n\nJava\n\nBuilding an application that allows users to post to search for tenants, or to find rooms and\n\ncontact through posts.\n\n4\n\nI am in responsible of creating the personal information area, managing login registration,\n\nadding, removing, changing, and renewing postings, as well as processing payments via the MoMo payment gateway (testing).\n\nFirebase realtime, Firebase storage, SQLite\n\nhttps://github.com/TranDatk/LTDD_CS2001_Nhom22\n\nhttps://github.com/Penz7/ecommerce\n\n-\n\ndemo\n\n-\n\nwebsite.github.io\n\nThe website is designed to simulate an online sales website using HTML/CSS and JavaScript technologies\n\nhttps://github.com/Penz7/clicnicsystem\n\nPython (Flask\n\n-\n\nadmin), HTML/CSS, Javascript\n\nSimulate a private clinic website with user login and registration services provided by Python, and utilize Flask admin to manage medical examination orders and drug payment invoices on\n\nthe doctor's and nurse's interfaces at that clinic.\n\nJava, Kotlin, and Javascript are the most popular uses.\n\nFirebase , SQL Server, MySQL, SQLite and Jetpack Compose (Android)\n\nCan read, understand, and communicate in basic English\n\n© topcv.vn"

lines = text_items.split('\n')

# Process the lines
for line in lines:
    # Perform desired operations on each line
    print(line)


Cap Tan Dat

Web developer intern

 0961414817 ✉ tandatkt002@gmail.com  https://github.com/Penz7

 Ho Chi Minh City,Ward 7,Binh Thanh District

OBJECTIVE

...

the doctor's and nurse's interfaces at that clinic.

Java, Kotlin, and Javascript are the most popular uses.

Firebase , SQL Server, MySQL, SQLite and Jetpack Compose (Android)

Can read, understand, and communicate in basic English

© topcv.vn